# Lesson 0 - Getting Started

## Overview

In this training, we aim to walk you through a hands-on calibration experiment with the Poconos watershed. Specifically, we will be calibrating the basin using observed and modeled streamflow at two USGS locations over a few months in 2011 and validate ove the remaining period in 2011. This is just for demonstration purposes and not the common practice on NWM calibration. In fact, the calibration of the NWM is done using a 5-year period in order to cover different climate regimes in the calibration period. Details of domains, calibration experiments for this exercise will be provided in this and the upcoming lessons. 

The calibration procedure relies on a python library named PyWrfHydroCalib, which explicitly handles the setup, model tuning, and provides diagnostics supporting the calibration process. This lesson is intended to provide an overview of few key definitions, system requirements, and orienting you through the calibration material provided to you. We will describe the calibration procedure and steps fundementals in Lesson 1 and follow up with hands-on experiment in Lesson 3 and 4 where we will setup, calibrate and validate a small domain.

## What Is It? Defining Key Terms:
### WRF-Hydro: 
An open-source community model, is used for a range of projects, including flash flood prediction, regional hydroclimate impacts assessment, seasonal forecasting of water resources, and land-atmosphere coupling studies. 

### The National Water Model (NWM):
A joint development effort between the National Center for Atmospheric Research (NCAR) and the National Weather Service (NWS) Office of Water Prediction (OWP). The NWM is a set of operational analysis and forecast configurations of the WRF-Hydro modeling system (Gochis et al., 2013) which runs over the entire continental United States (https://water.noaa.gov/about/nwm) as well as Hawaii and Puerto Rico. Alaska will also become operational in NWMv30. 

### PyWrfHydroCalib:
A robust workflow for calibrating parameters over individual basins with a focus on automating the process for hundreds of forecast points in the NWM domain. Here, we describe the various steps involved in setting up a WRF-Hydro calibration experiment, along with how to execute the calibration workflow and access the various results from it. This version of the calibration workflow is the RnD branch of the NWMv30 calibration and it not yet finalized and official. There are still developments and code improvement underway and it will be finalized before our official NWMV30 onborading, which is scheduled for Jan 2022. 

## PyWrfHydroCalib System Requirements
Calibration goes beyond setting up the NWM or WRF-Hydro for a model simulation. Some understanding of the preparation of datasets, hydrologic parameters, and their associated impact on model states is needed. You will decide which parameters to calibrate, along with the proper ranges, and the length of the calibration simulation/evaluation period. It is highly encouraged that you run sensitivity analysis for the modeling domain to explore which parameters have significant impact on the modeled states being calibrated. 


### Software and conventions
#### Python Packages:
To run the calibration workflow, there are multiple software requirements and pre-processing steps that need to take place prior to actual calibration. Currently, the calibration workflow relies on a set of Python and R libraries to execute various functions and plotting capabilities. This is in addition to existing requirements to compile and run WRF-Hydro/NWM. As of early 2019, the calibration workflow code has been migrated from Python 2 to Python 3. The following Python libraries are used by the workflow:

1.) numpy
2.) NetCDF4 (for Python)
3.) pandas
4.) psutil
5.) psycopg2
6.) xarray


In [ ]:
# Optional: Verify Python Libraries
import numpy
import netCDF4
import pandas
import psutil
import psycopg2
import xarray
print('all python packages accounted for')

#### R Libraries
For R, there are several additional libraries required for proper analysis and plotting to take place. The following R libraries are needed for proper execution:

1.) data.table
2.) ggplot2
3.) ncdf4
4.) plyr
5.) boot
6.) sensitivity
7.) randtoolbox
8.) gridExtra
9.) hydroGOF
10.) qmap 
11.) zoo  
12.) raster 

In [5]:
# Verify R libraries (for Jupyter notebook on Cheyenne, switch to R kernel for now, will be updated for docker notebook)
library(data.table)
library(ggplot2)
library(ncdf4)
library(plyr)
library(gridExtra)
library(hydroGOF)
library(qmap)
library(zoo)
library(raster)
library(boot)        # used for the sensitivity analysis 
library(sensitivity) # used for the sensitivity analysis 
library(randtoolbox) # used for the sensitivity analysis 

ERROR: Error in library(data.table): there is no package called ‘data.table’


### Prerequisites:
1. Installation of the WRF-Hydro/NWM modeling system (use a version that is pointed out in the release notes of the package) See the 'WRF-Hydro Technical Description and User Guide' available from the WRF-Hydro Modeling System website for details: https://ral.ucar.edu/projects/wrf_hydro. 

1.  Preparation of a modeling domain. 

1.  Preparation of associated forcing files necessary to run the model simulations. 

1.  Preparation of input observation files that will be used in the calibration workflow. Further detail regarding required input files can be found in the PyWrfHydroCalib can be found in the User Manual. 

**--> All of these files are provided for you in this training. We have provided the WRF-Hydro code that has already compiled. There is also a supllemental lesson (Lesson-S1-wrf_hydro_compile.ipynb) that walk you through the model compilation if you wish to do so.**

### Which files will be updated during calibration

During the calibration process we will be updating that parameters that exist in one of the followign files. 

*soil_properties.nc* :
A 2D property file that contains values impacting sub-surface hydrologic response.

*HYDRO_TBL_2D.nc* :
The 2D NetCDF of surface hydrologic parameters impacting hydrologic response for overland flow routing.

*Fulldom.nc* :
The 2D geospatial fabric utilized for the high-resolution routing.

*GWBUCKPARM.nc* : 
Groundwater bucket parameter file.

These files are located in the domain directory that are prepared for you, and will be discussed in details later. 


## Orientation to training material

All the required codes and domain files are being prepared for you in advance. Let s take a look at the content of the materials. 


In [5]:
%%bash
cd /home/docker
ls

example_case
lessons
miniconda3
PyWrfHydroCalib
wrf_hydro_nwm_public


**wrf_hydro_nwm_public**: this directory has the WRF_Hydro code that is already compiled for you. Refer to supplemntal lesson (Lesson-S1-wrf_hydro_compile.ipynb) for details on how to compile the model. Lets take a look at the content of the `Run` folder that is created after model compilations and contains the model exectuable as well as namelists and table files that are compatiable with the compiled version of the code. *Note* that we will use the TBL files from this `Run` directory, however, the namelists will be generated by the python workflow (PyWrfHydroCalib). 

In [ ]:
%%bash
ls /home/docker/example_case/wrf_hydro_nwm_public/trunk/NDHMS/Run

In [4]:
%%bash
ls /home/docker/example_case/Calibration/Input_Files/01447720/

FORCING
Fulldom.nc
geo_em.nc
GEOGRID_LDASOUT_Spatial_Metadata.nc
GWBUCKPARM.nc
HYDRO_TBL_2D.nc
LAKEPARM.nc
OBS
params.txt
RouteLink.nc
RouteLink.reExpFrom.Rdb
RouteLink.reExp.nc
RouteLink.reExpTo.Rdb
RouteLink.reInd.Rdb
script_forcing_subset.txt
script_forcing_subset.txt_NWM_REALTIME
soil_properties.nc
spatialweights.nc
wrfinput.nc


### Navigating the Example File Structure:

Within the main Directory, there are 3 subdirectories:

example_case -- contains all domain and input files:


In [6]:
%%bash
cd /home/docker/example_case/Calibration/Input_Files/01447720/
ls

FORCING
Fulldom.nc
geo_em.nc
GEOGRID_LDASOUT_Spatial_Metadata.nc
GWBUCKPARM.nc
HYDRO_TBL_2D.nc
LAKEPARM.nc
OBS
params.txt
RouteLink.nc
RouteLink.reExpFrom.Rdb
RouteLink.reExp.nc
RouteLink.reExpTo.Rdb
RouteLink.reInd.Rdb
script_forcing_subset.txt
script_forcing_subset.txt_NWM_REALTIME
soil_properties.nc
spatialweights.nc
wrfinput.nc


PyWrfHydroCalib  -- contains calibration scripts, and user setup files

In [7]:
%%bash
cd /home/docker/PyWrfHydroCalib/
ls

calibOrchestrator.py
calib.py
core
initDB.py
inputDomainMeta.py
jobInit.py
LICENSE.md
PyWrfHydroCalib_User_Guide.pdf
README.md
runValidOrchestrator.py
sensitivity.py
setup_files
spinOrchestrator.py
spinup.py
util
validation.py
validOrchestrator.py


In [8]:
%%bash
cd /home/docker/PyWrfHydroCalib/setup_files/
ls

calib_params.tbl
domainMetaTemplate.csv
gage_list_template.csv
sens_params.tbl
_setup.parm
setup.parm
untitled.txt


## Conclusion:

Once all Input and Setup files are prepared, and verified, we can begin the process of setting up a calibration. Proceed to *Lesson 1 - Calibration Overview*